# Esta tarea fue realizada en equipo.
# Integrantes del equipo: 
# Betsy Torres & Mariana Briones

In [2]:
# Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# importamos los datos 
data_oil=pd.read_excel('EXAMEN_P1.xlsx')
data_oil

,Year 1,Year 2,Year 3,Year 4,Year 5
0,342299,210646,267698,304886,50162
1,224626,244301,398344,230886,49800
2,64300,202526,146428,290904,50120
3,127986,326343,286020,273425,49907
4,402335,367261,398079,226830,50080
...,...,...,...,...,...
2995,184015,262116,377285,199294,50198
2996,194969,393200,157290,225308,49912
2997,172223,374372,398071,187111,50033
2998,18656,459320,217392,212004,50120


In [5]:
# Generamos Data Frame de los flujos por medio de compresión de listas.

data = pd.DataFrame(columns=['y0', 'y1', 'y2', 'y3','y4','y5'], index=range(3000))
data['y0']= -1800000
data['y1']=[i*(10-8)-40000 for i in data_oil.iloc[:,0]]
data['y2']=[i*(10-8)-40000 for i in data_oil.iloc[:,1]]
data['y3']=[i*(10-8)-40000 for i in data_oil.iloc[:,2]]
data['y4']=[i*(10-8)-40000 for i in data_oil.iloc[:,3]]
data['y5']=[i*(10-8)-40000 for i in data_oil.iloc[:,4]]
data

,y0,y1,y2,y3,y4,y5
0,-1800000,644598,381292,495396,569772,60324
1,-1800000,409252,448602,756688,421772,59600
2,-1800000,88600,365052,252856,541808,60240
3,-1800000,215972,612686,532040,506850,59814
4,-1800000,764670,694522,756158,413660,60160
...,...,...,...,...,...,...
2995,-1800000,328030,484232,714570,358588,60396
2996,-1800000,349938,746400,274580,410616,59824
2997,-1800000,304446,708744,756142,334222,60066
2998,-1800000,-2688,878640,394784,384008,60240


In [6]:
# Generamos funcion para calcular tie y npv

def func(rf:'Tasa libre de riesgo', flujos:'flujos de efectivo'):
    '''Esta función calculará la TIE y el VPN de los datos, los cuales son fujos 
    de efectivo, y arrojará los resultados por medio de listas'''
    l_irr=[]
    l_npv=[]

    for i in range(len(flujos)):
        ans_irr=np.irr(flujos.iloc[i,0:6])*100
        ans_npv=np.npv(rf,flujos.iloc[i,0:6])
        
        l_irr.append(ans_irr)        
        l_npv.append(ans_npv)
    
    return l_irr,l_npv

# Listas de la tie y el valor presente neto
tie, vpn = func(0.03,data)

<ipython-input-6-20952a9375dc>:10: DeprecationWarning: numpy.irr is deprecated and will be removed from NumPy 1.20. Use numpy_financial.irr instead (https://pypi.org/project/numpy-financial/).
  ans_irr=np.irr(flujos.iloc[i,0:6])*100
<ipython-input-6-20952a9375dc>:11: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
  ans_npv=np.npv(rf,flujos.iloc[i,0:6])


In [7]:
# Data frame completo de los flujos, valor pesente neto y tie
data['IRR']=tie
data['NPV']=vpn
data['NPV']=data['NPV'].round(2)
data['IRR']=data['IRR'].round(2)
data.head()

,y0,y1,y2,y3,y4,y5,IRR,NPV
0,-1800000,644598,381292,495396,569772,60324,7.43,196856.15
1,-1800000,409252,448602,756688,421772,59600,5.98,138809.60
2,-1800000,88600,365052,252856,541808,60240,-9.63,-605132.06
3,-1800000,215972,612686,532040,506850,59814,2.50,-23985.42
4,-1800000,764670,694522,756158,413660,60160,19.40,708469.44


In [12]:
data['NPV'].mean(), data['NPV'].std()

(-27823.859093333293, 372903.58182924957)

1. What the probability that the IRR is over the Risk free rate?

In [13]:
rf=0.03
c1 = [n for n in data['NPV'] if n>1000000]
ans1=len(c1)/len(data)
print(ans1*100,'%')

0.0 %


2. What the probability that the IRR is over the 35%?

In [13]:
c2 = [n for n in data['IRR'] if n>35]
ans2=len(c2)/len(data)
print(ans2*100,'%')

53.55 %


3. What the probability that the project value is over $2M?

In [11]:
c3 = [n for n in data['NPV'] if n>2000000]
ans3 = len(c3)/len(data)
print(ans3*100,'%')

40.400000000000006 %


4. What’s the probability that the IRR is between 10% and 20%?

In [12]:
c4 = [n for n in data['IRR'] if 10<n<20]
ans4 = len(c4)/len(data)
print(ans4*100,'%')

12.0 %


In [11]:
# Función propuesta para obtener flujos

def flujos(data:'Datos', p:'Precio', cost:'Costo de producción',
           fix:'Costos fijos', inv_inicial:'Inversión inicial'):
    '''
    Con esta función calculará los flujos y los deplegará en un data frame
    Datos: cantidad de ventas de n años
    Precio: Precio unitario
    Costo: Costo producción expresado en porcentaje
    Fix: Costos fijos expresados de manera positiva (sin signo)
    Inversión inicial: Cifra de la inversión inicial expresada en positivo(sin signo)
    '''
    
#     l_años= ['y'+str(n) for n in range(len(data.columns))]
    
    flows = pd.DataFrame(columns=['y0', 'y1', 'y2', 'y3','y4','y5' ], index=range(len(data)))
    
    flows['y0']= -inv_inicial
    flows['y1']=[i*(p-cost)-fix for i in data.iloc[:,0]]
    flows['y2']=[i*(p-cost)-fix for i in data.iloc[:,1]]
    flows['y3']=[i*(p-cost)-fix for i in data.iloc[:,2]]
    flows['y4']=[i*(p-cost)-fix for i in data.iloc[:,3]]
    flows['y5']=[i*(p-cost)-fix for i in data.iloc[:,4]]
    
#     for j in range(len(data.columns)):
# #         flows[str(flows.columns[j+1])]=[i*(p-cost)-fix for i in data.iloc[:,j]]
#         flows[str(flows.columns[j+1])]=[i*(p-cost)-fix for i in [data.iloc[:,a] for a in range(len(data.columns))]]

    
    return flows

In [12]:
flujos_func=flujos(data_oil,5,0.8,40000,1800000)
flujos_func

,y0,y1,y2,y3,y4,y5
0,-1800000,815649.2,1135869.8,1914272.6,801869.0,364539.8
1,-1800000,1068149.0,1104243.8,-30793.6,799584.2,-40000.0
2,-1800000,76449.2,1366416.2,305513.0,799806.8,232823.6
3,-1800000,1193964.2,842491.4,1115365.4,799722.8,-40000.0
4,-1800000,1208739.8,978789.8,1909207.4,802511.6,553397.0
...,...,...,...,...,...,...
1995,-1800000,957756.2,1095969.8,444222.2,799189.4,216414.2
1996,-1800000,869421.8,1056221.0,321313.4,802142.0,-40000.0
1997,-1800000,1011163.4,887784.2,1980779.6,799160.0,-29193.4
1998,-1800000,1014414.2,1002171.2,1557680.0,800420.0,481677.8
